# Federated Cluster Model with Flex  for Anomaly Detection

In this notebook we show how to use an Cluster model for anomaly detection with federated learning using the flexible

First we do all the imports needed.

In [ ]:
from flexanomalies.utils import ClusterAnomaly
from flexanomalies.utils.load_data import load_and_split_dot_mat, federate_data
from flexanomalies.pool.aggregators_cl import aggregate_cl
from flexanomalies.pool.primitives_cluster import (
    build_server_model_cl,
    copy_model_to_clients_cl,
    train_cl,
    set_aggregated_weights_cl,
    get_clients_weights_cl,
)
from flexanomalies.utils.save_results import save_experiments_results
from flex.pool import FlexPool

Load data, define model parameters and define model

In [2]:
model_params = {"n_clusters": 4, "contamination": 0.1}

In [3]:
X_train, X_test, y_train, y_test = load_and_split_dot_mat(
    "flexanomalies/datasets/data/shuttle.mat", 0.3
)

In [4]:
model = ClusterAnomaly(**model_params)

Once the data  is loaded, we have to federate it. For this we use the FLEX library. There are two ways to federate the data, using an IID distribution or a non IID distribution. For the IID distribution we can use the ìid_distribution function of 

FedDataDistribution. If we use a non-IID distribution, it is necessary to use a custom configuration, such as the one used in the federate_data function. For more information, go to the FLEX library notebooks, and take a look at the Federate Data with 

FLEXible notebooks.

## Creating the federated architecture

When creating the federated architecture, we use FlexPool. Since we are running a client-server architecture, we use the client_server_architecture function. 

In [5]:
flex_dataset = federate_data(5, X_train, y_train)

In [6]:
pool = FlexPool.client_server_pool(
    fed_dataset=flex_dataset,
    server_id="cluster_server",
    init_func=build_server_model_cl,
    model=model,
)

## Run the federated learning experiment and Evaluate


Now, we can run the federated experiment for multiple rounds using the decorators. 

Once the model is trained, we need to evaluate it at the server level.


In [ ]:
for i in range(3):
    print(f"\nRunning round: {i}\n")
    pool.servers.map(copy_model_to_clients_cl, pool.clients)
    pool.clients.map(train_cl)
    pool.aggregators.map(get_clients_weights_cl, pool.clients)
    pool.aggregators.map(aggregate_cl, model=model)
    pool.aggregators.map(set_aggregated_weights_cl, pool.servers)
output_model = pool.servers._models["cluster_server"]["model"]
output_model.evaluate(X_test, y_test)
print()

In [ ]:
output_model.result_metrics_

In [11]:
save_experiments_results(
    "cluster",
    output_model,
    "test_cluster_notebook",
    model_params,
    "shuttle.mat",
    5,
    3,
    0.3,
)

## End of Notebook